In [3]:
import torch
import pandas as pd
import json

In [4]:
nodes_df = pd.read_pickle('experiment_runs/run_2025-09-26_21-25-12/nodes_df.pkl')
edges_df = pd.read_pickle('experiment_runs/run_2025-09-26_21-25-12/edges_df.pkl')

In [51]:
task_to_idx = json.load(open('experiment_runs/run_2025-09-26_21-25-12/task_to_idx.json', 'r'))
idx_to_task = {value: key for key, value in task_to_idx.items()}
task_to_idx

{'text-generation': 0,
 'question-answering': 1,
 'text-to-video': 2,
 'image-to-video': 3,
 'image-to-3d': 4,
 'robotics': 5,
 'translation': 6,
 'feature-extraction': 7,
 'text-to-3d': 8,
 'text-to-speech': 9,
 'automatic-speech-recognition': 10,
 'image-classification': 11,
 'table-question-answering': 12,
 'fill-mask': 13,
 'multiple-choice': 14,
 'visual-question-answering': 15,
 'summarization': 16,
 'image-to-text': 17,
 'image-feature-extraction': 18,
 'text-to-image': 19,
 'text-to-audio': 20,
 'reinforcement-learning': 21,
 'image-text-to-text': 22,
 'text-classification': 23,
 'sentence-similarity': 24,
 'zero-shot-classification': 25,
 'text-retrieval': 26,
 'token-classification': 27,
 'object-detection': 28,
 'audio-classification': 29,
 'image-segmentation': 30,
 'time-series-forecasting': 31,
 'video-classification': 32,
 'zero-shot-image-classification': 33,
 'any-to-any': 34,
 'image-to-image': 35,
 'depth-estimation': 36,
 'tabular-classification': 37,
 'tabular-regr

In [13]:
nodes_df.head()

,id,description,createdAt,type,y_multi_lab,relationships,y
0,tencent/SRPO,---\nlibrary_name: diffusers\nlicense: other\n...,2025-09-08T12:44:15+00:00,model,[19],model_finetune_model:rockerBOO/flux.1-dev-SRPO...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,baidu/ERNIE-4.5-21B-A3B-Thinking,---\nlicense: apache-2.0\nlanguage:\n- en\n- z...,2025-09-08T14:18:31+00:00,model,[0],model_finetune_model:unsloth/ERNIE-4.5-21B-A3B...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Qwen/Qwen3-Next-80B-A3B-Instruct,---\nlibrary_name: transformers\nlicense: apac...,2025-09-09T15:40:56+00:00,model,[0],model_finetune_model:unsloth/Qwen3-Next-80B-A3...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Qwen/Qwen3-Next-80B-A3B-Thinking,---\nlibrary_name: transformers\nlicense: apac...,2025-09-09T15:45:31+00:00,model,[0],model_finetune_model:kikekewl/Qwen3-Next-80B-A...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,google/embeddinggemma-300m,---\nlicense: gemma\npipeline_tag: sentence-si...,2025-07-17T19:53:55+00:00,model,"[24, 7]",model_finetune_model:sigridjineth/colbert-ko-e...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
edges_df.head()

,source_node,dest_node,edge_type,edge_attr
match,,,,
0,tencent/SRPO,rockerBOO/flux.1-dev-SRPO,model_finetune_model,0
1,tencent/SRPO,wikeeyang/SRPO-Refine-Quantized-v1.0,model_quantized_model,3
1,tencent/SRPO,befox/SRPO-GGUF,model_quantized_model,3
1,tencent/SRPO,wikeeyang/SRPO-for-ComfyUI,model_quantized_model,3
0,baidu/ERNIE-4.5-21B-A3B-Thinking,unsloth/ERNIE-4.5-21B-A3B-Thinking,model_finetune_model,0


In [7]:
f = open('experiment_runs/run_2025-09-26_21-25-12/old_to_new_idx.json','r')

old_to_new_idx_dict = json.load(f)

In [10]:
graph = torch.load('experiment_runs/run_2025-09-26_21-25-12/final_graph.pt')

In [11]:
graph

Graph(x=[389066, 768], y=[389066, 54], train_mask=[389066], val_mask=[389066], test_mask=[389066], edge_index=[2, 124056], edge_attr=[124056])

In [37]:
graph.nodes()

tensor([     0,      1,      2,  ..., 389063, 389064, 389065])

In [39]:

new_to_old_idx = {value: int(key) for key, value in old_to_new_idx_dict.items()}
def get_ori_node(new_node_idx):
    ori_id = new_to_old_idx[new_node_idx]
    return ori_id, nodes_df.iloc[ori_id].to_dict()
    

### Example check 1

In [40]:
ori_id, ori_record = get_ori_node(389065)

In [62]:
task_labs = [idx_to_task[idx] for idx in ori_record["y_multi_lab"]]
print(f''' 
    id = {ori_record["id"]} , 
        type={ori_record["type"]}, task_lab_idx={ori_record["y_multi_lab"]}, {task_labs}. its original ID is {ori_id} in nodes_df.
        
    ''')

 
    id = kjydb/cafe_010 , 
        type=dataset, task_lab_idx=[5], ['robotics']. its original ID is 389574 in nodes_df.

    


In [56]:
nodes_df[nodes_df['id']=='kjydb/cafe_010']

,id,description,createdAt,type,y_multi_lab,relationships,y
389574,kjydb/cafe_010,---\nlicense: apache-2.0\ntask_categories:\n- ...,2025-09-17T02:31:25+00:00,dataset,[5],,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [60]:
edges_df[(
    (edges_df['source_node']=='kjydb/cafe_010')| 
    (edges_df['dest_node']=='kjydb/cafe_010')
)]

,source_node,dest_node,edge_type,edge_attr
match,,,,


In [75]:
graph.degrees()[389065]

tensor(0.)

=> So this is correct that this node had no edges, and its task label is robotics

### Example check 2

In [124]:
ori_id, ori_record = get_ori_node(12)

In [125]:
task_labs = [idx_to_task[idx] for idx in ori_record["y_multi_lab"]]
print(f''' 
    id = {ori_record["id"]} , 
        type={ori_record["type"]}, task_lab_idx={ori_record["y_multi_lab"]}, {task_labs}. its original ID is {ori_id} in nodes_df.
        
    ''')

 
    id = inclusionAI/Ling-mini-2.0 , 
        type=model, task_lab_idx=[0], ['text-generation']. its original ID is 12 in nodes_df.

    


In [126]:
nodes_df[nodes_df['id']=='inclusionAI/Ling-mini-2.0']

,id,description,createdAt,type,y_multi_lab,relationships,y
12,inclusionAI/Ling-mini-2.0,---\nlicense: mit\nbase_model:\n- inclusionAI/...,2025-09-08T13:43:44+00:00,model,[0],model_quantized_model:mlx-community/Ling-mini-...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [127]:
edges_df[(
    (edges_df['source_node']=='inclusionAI/Ling-mini-2.0') | 
    (edges_df['dest_node']=='inclusionAI/Ling-mini-2.0')
)]

,source_node,dest_node,edge_type,edge_attr
match,,,,
0,inclusionAI/Ling-mini-2.0,mlx-community/Ling-mini-2.0-8bit,model_quantized_model,3


In [128]:
graph.degrees()[12]

tensor(1.)

# Double check from original

Notice from the original page on huggingface, https://huggingface.co/mlx-community/Ling-mini-2.0, there are total of 16 models, but the scraping script only downloaded one. Observe that from the 16 quantized models, there is only 1 with last modified on `September 11, 2025` from the link.

(Checked on 26th Sept)


In [ ]:
temp = pd.read_json('../HuggingKG_V20250916155543/model_quantized_model.json')

In [94]:
temp[temp['base_model_id']=='inclusionAI/Ling-mini-2.0']

,base_model_id,model_id
752,inclusionAI/Ling-mini-2.0,mlx-community/Ling-mini-2.0-8bit


In [ ]:
temp = pd.read_json('../HuggingKG_V20250916155543/models.json')

In [96]:
temp[temp['id']=='inclusionAI/Ling-mini-2.0']

,id,name,createdAt,lastModified,downloads,likes,region,other,libraries,license,languages,pipeline_tag,description
13,inclusionAI/Ling-mini-2.0,Ling-mini-2.0,2025-09-08T13:43:44+00:00,2025-09-12T03:10:29+00:00,907,118,🇺🇸 Region: US,"[bailing_moe, conversational, custom_code, aut...","[Transformers, Safetensors]",mit,[],[Text Generation],---\nlicense: mit\nbase_model:\n- inclusionAI/...
